In [1]:
from __future__ import print_function
import subprocess
import sys
import os
from sys import stdout
from time import sleep
from astropy import units as u
from astropy.coordinates import SkyCoord


In [ ]:
#WHICH NODES IS BREAKTHROUGH LISTEN OBSERVING WITH (BANKS):

banks = {}
for x in range(0,8):
    banks["blc0" + str(x)] = subprocess.check_output(["redis-cli",
                                                        "mget", "blc0" +
                                                        str(x) + "_observing"])
for x in range(10,18) + range(20,28) + range(30,38):
    banks["blc" + str(x)] = subprocess.check_output(["redis-cli",
                                                        "mget", "blc" +
                                                        str(x) + "_observing"])
#generates bank key with observation status as value

os.system("rm /home/jkyle/blcbanks/banks.txt")
#remove created text file to ensure overwrite with new status

for x in banks:
    if banks[x] == "on\n":
        nodes = []
        nodes.append(x.upper())
    else:
        continue
    for x in nodes:
        message = ("echo 'nodes,host=" + x.upper() +
                   ",region=us value=1' >> /home/jkyle/blcbanks/banks.txt")
        os.system(message)
#formatting for influxdb entry

if all(x == 'off\n' for x in banks.values()):
    message = "Breakthrough Listen is not observing.\n"
    message = ("echo 'nodes,host=Breakthrough\\ Listen\\ is\\ not\\ observing," +
                "region=us value=0' > /home/jkyle/blcbanks/banks.txt")
    os.system(message)
#formatting for influxdb entry

#os.system("scp /home/jkyle/blcbanks/banks.txt jkyle@104.199.127.246:/home/jkyle/public_html/")
#transferrnig to virtual machine redis server


In [ ]:
#GENERATES RAW FILES AND TELESCOPE DATA (GRAFANA):

os.system("rm /home/jkyle/grafana.txt")

bank0 = ('00', '01', '02', '03', '04', '05', '06', '07',
         '10', '11', '12', '13', '14', '15', '16', '17',
         '20', '21', '22', '23', '24', '25', '26', '27',
         '30', '31', '32', '33', '34', '35', '36', '37')

for x in bank0:
    try:
        y = ("BLC" + x)
        i = subprocess.check_output(["/home/jkyle/GBT-ADQ/mysql/blc/blc", ("blc" + x)])
        if i == i:
                os.system("echo '" + i[0:5].strip("\n") + "raw,host=" +
                        i[5:].strip("\n") + ",region=us value=1' >> grafana.txt")
    except:
        continue
        #this will print an error code, not sure what influxdb will do

RA = subprocess.check_output("/home/jkyle/GBT-ADQ/mysql/show_mysql_ra")
os.system("echo '" + "ra,host=gbt,region=us value="
                + str(RA[0:8].strip("\n")) + "' >> grafana.txt")

DEC = subprocess.check_output("/home/jkyle/GBT-ADQ/mysql/show_mysql_dec")
os.system("echo '" + "dec,host=gbt,region=us value="
                + str(DEC[0:8].strip("\n")) + "' >> grafana.txt")


RAhms = subprocess.check_output("/home/jkyle/GBT-ADQ/mysql/show_mysql_ra")
DEChms = subprocess.check_output("/home/jkyle/GBT-ADQ/mysql/show_mysql_dec")
RAhms = float(RA.strip("\n"))
DEChms = float(DEC.strip("\n"))
coordinates = SkyCoord(ra=RAhms*u.degree, dec=DEChms*u.degree)
coordinates = coordinates.to_string("hmsdms")
RAhms = coordinates[0:13]
DEChms = coordinates[14:]
#need to convert to HH:MM:SS

os.system("echo '" + "rahms,host=gbt,region=us value="
                + "\"" + str(RAhms) + "\"" +  "' >> grafana.txt")
os.system("echo '" + "dechms,host=gbt,region=us value="
                + "\"" + str(DEChms) + "\"" +  "' >> grafana.txt")
#need to convert to DD:MM:SS

SOURCE = subprocess.check_output("/home/jkyle/GBT-ADQ/mysql/show_mysql_source")
os.system("echo '" + "source,host=" + SOURCE.strip("\n")
                + ",region=us value=1" + "' >> grafana.txt")

DISH = subprocess.check_output("/home/jkyle/GBT-ADQ/mysql/show_mysql_ant")
os.system("echo '" + "dish,host=" + DISH.strip("\n")
                + ",region=us value=1" + "' >> grafana.txt")

DATE = subprocess.check_output("/home/jkyle/GBT-ADQ/mysql/show_mysql_date")
os.system("echo '" + "date,host=" + DATE.strip("\n")
                + ",region=us value=1" + "' >> grafana.txt")

DIR = subprocess.check_output("/home/jkyle/GBT-ADQ/mysql/show_mysql_data")
os.system("echo '" + "dir,host=" + DIR.strip("\n")
                + ",region=us value=1" + "' >> grafana.txt")

REM = subprocess.check_output("/home/jkyle/GBT-ADQ/mysql/show_mysql_remaining")
os.system("echo '" + "rem,host=" + REM.strip("\n")
                + ",region=us value=1" + "' >> grafana.txt")

os.system("scp /home/jkyle/grafana.txt jkyle@104.199.127.246:/home/jkyle/public_html/")


In [ ]:
#PULL INPUT DATA FROM COMPUTE NODES (IOSTATUS):

os.system("~obs/bin/all_check_iostats > iostats")
stats = open("iostats")
data = [line.split() for line in stats.readlines()]

os.system("rm iostats.txt")

In [129]:
test = range(0,12,4)
print(test)

[0, 4, 8]


In [127]:
write = {}
for x in range(0,128,4):
    i = x+2
    ii = x+3
    write[str(data[x][0]) + "\\ sda"] = data[i][5]
    write[str(data[x][0]) + "\\ sdb"] = data[ii][5]

for key in sorted(write.iterkeys()):
    if float(write[key]) == 0.00:
        pass
    else:
        os.system("echo '%s,host=gbt,location=wkbs value=%s' >> iostats.txt" % (key, write[key]))

read = {}
for x in range(0,128,4):
    i = x+2
    ii = x+3
    read[str(data[x][0]) + "\\ sda"] = data[i][6]
    read[str(data[x][0]) + "\\ sdb"] = data[ii][6]
    
for key in sorted(read.iterkeys()):
    if float(read[key]) == 0.00:
        pass
    else:
        os.system("echo '%s,host=gbt,location=rkbs value=%s' >> iostats.txt" % (key, read[key]))

util = {}
for x in range(0,128,4):
    i = x+2
    ii = x+3
    util[str(data[x][0]) + "\\ sda"] = data[i][-1]
    util[str(data[x][0]) + "\\ sdb"] = data[ii][-1]
    
for key in sorted(util.iterkeys()):
    if float(util[key]) == 0.00:
        pass
    else:
        os.system("echo '%s,host=gbt,location=util value=%s' >> iostats.txt" % (key, util[key]))

os.system("scp /home/jkyle/iostats.txt jkyle@104.199.127.246:/home/jkyle/public_html/")

echo 'blc00\ sda,host=gbt,location=wkbs value=427834.00' >> iostats.txt
echo 'blc01\ sdb,host=gbt,location=wkbs value=547870.00' >> iostats.txt
echo 'blc02\ sdb,host=gbt,location=wkbs value=657804.00' >> iostats.txt
echo 'blc03\ sdb,host=gbt,location=wkbs value=439998.00' >> iostats.txt
echo 'blc04\ sda,host=gbt,location=wkbs value=683898.00' >> iostats.txt
echo 'blc05\ sda,host=gbt,location=wkbs value=573754.00' >> iostats.txt
echo 'blc06\ sdb,host=gbt,location=wkbs value=617464.00' >> iostats.txt
echo 'blc07\ sda,host=gbt,location=wkbs value=542556.00' >> iostats.txt
echo 'blc10\ sda,host=gbt,location=wkbs value=618236.00' >> iostats.txt
echo 'blc11\ sda,host=gbt,location=wkbs value=573522.00' >> iostats.txt
echo 'blc12\ sda,host=gbt,location=wkbs value=683512.00' >> iostats.txt
echo 'blc13\ sda,host=gbt,location=wkbs value=527758.00' >> iostats.txt
echo 'blc14\ sda,host=gbt,location=wkbs value=496126.00' >> iostats.txt
echo 'blc15\ sdb,host=gbt,location=wkbs value=531768.00' >> iost

0